In [1]:
%run GameInterface.ipynb

In [2]:
import math

class Node:
    
    def __init__(self, move, parent):
        
        self.move = move
        self.parent = parent
        self.children = []
        self.value = 0
        self.visits = 0
        
        # value multiplier
        self.current_player_val = 1


    # generate the children of this node
    # state    - A copy of the game represented by this node.
    # positive - True if these nodes represent MCTS player's choices.
    def expand_node(self, state):
        if not state.isGameOver():
            for move in state.getMoves():
                child = Node(move, self)
                self.children.append(child)

    def update(self, value):
        self.visits = self.visits + 1
        self.value = self.value + value * self.current_player_val
    
    def is_leaf(self):
        return len(self.children)==0

    def has_parent(self):
        return self.parent is not None
    
    def ucb1(self):
        if self.parent is None:
            return 0
        if self.visits == 0:
            return 9999 # infinity
        return self.parent.current_player_val * self.current_player_val * self.winRatio() + \
                    math.sqrt(2*math.log(self.parent.visits) / float(self.visits))
    
    def winRatio(self):
        if self.visits == 0:
            return 0
        return self.value / float(self.visits)
    

class MCTSPlayer(Player):
        
    def __init__(self, piece, rollouts=500, max_rollout=100):
        super(MCTSPlayer, self).__init__(piece)
        self.rollouts = max(1, rollouts)
        self.max_rollout = max(1, max_rollout)
        self.root_node  = Node(None, None)
        self.last_state = None
        self.last_rollout_state = None
        
    # returns child of current_node with best UCB1 score
    def select_child(self, current_node):
        best_child = current_node.children[0]
        for child in current_node.children:
            if child.ucb1() > best_child.ucb1():
                best_child = child
        return best_child    

    def getMoveValues(self, game):
        
        
        for i in range(self.rollouts):
            
            # selection
            current_node = self.root_node
            current_state = game.clone()
            while not current_node.is_leaf():
                current_node = self.select_child(current_node)
                current_state.update(current_node.move)
            if self.player_piece!=current_state.current_player_val:
                current_node.current_player_val = -1

            #expansion
            current_node.expand_node(current_state)
            if not current_node.is_leaf():
                current_node = self.select_child(current_node)
                current_state.update(current_node.move)
                if self.player_piece!=current_state.current_player_val:
                    current_node.current_player_val = -1
            # save selected state for drawing
            self.last_state = current_state.clone()
            
            # rollout
            depth = 0
            while not current_state.isGameOver() and depth<self.max_rollout:
                moveList = current_state.getMoves()
                move = random.randint(0,len(moveList)-1)
                current_state.update(moveList[move])
                depth += 1
            result = self.evaluate(current_state)
            # save simulated end state for drawing
            self.last_rollout_state = current_state.clone()
            
            # backpropagate
            while current_node.has_parent():
                current_node.update(result)
                current_node = current_node.parent
            current_node.update(result)                       
        # return all of roots children
        return self.root_node.children
    
    def chooseMove(self, game):
        
        children = self.getMoveValues(game)
            
        # select best child from root
        best_child = children[0]
        for child in children:
            if child.winRatio() > best_child.winRatio():
                best_child = child
                
        # return move of best child
        return best_child.move